In [4]:
# 選手情報・過去レース情報から3連単舟券120種をクラス分類する
# biasだけに集約されてしまってうまくいかない。。

# 汎用ライブラリのimport
import sys
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math
import tensorflow as tf

In [5]:
# 自作ライブラリのimport
if os.environ['BR_HOME']+"/boatrace" not in sys.path:
    sys.path.append(os.environ['BR_HOME']+"/boatrace")
#print(sys.path)

from setup.myUtil import dbHandler

In [6]:
# 分析期間の指定は一旦ここでまとめてみる。
trainStartDate="20180101"
trainEndDate="20180531"
testStartDate="20180401"
testEndDate="20180430"

In [7]:
dbh=dbHandler.getDBHandle()
#dbHandler.closeDBHandle(dbh)

In [8]:
# trainの元データを取得
with dbh.cursor() as cursor:
    sel_sql = "select * from raceabst_forml_v \
               where raceDate between '%s' and '%s' \
               order by raceId "\
               % (trainStartDate,trainEndDate)
    cursor.execute(sel_sql)
    loadList=cursor.fetchall()
print("traindata:",len(loadList))

with dbh.cursor() as cursor:
    sel_sql = "select * from raceabst_forml_v \
               where raceDate between '%s' and '%s' \
               order by raceId "\
               % (testStartDate,testEndDate)
    cursor.execute(sel_sql)
    testLoadList=cursor.fetchall()
print("testdatta:",len(testLoadList))

traindata: 22761
testdatta: 4449


In [9]:
df = pd.io.json.json_normalize(loadList)
df.head()

,funaken,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1rank,l1score,l2Fcnt,l2boat2r,...,l6boat3r,l6motor2r,l6motor3r,l6rank,l6score,odds,raceDate,raceId,raceWaveHeight,raceWindSpeed
0,2-1-3,1,0.3636,0.5152,0.3558,0.5092,A1,13.871549,0,0.4746,...,0.4545,0.5421,0.6789,B2,0.851669,30.9,2018-01-01,20180101-06-01,3,5.0
1,2-1-6,0,0.2462,0.4462,0.3466,0.4830,B1,3.409304,0,0.3636,...,0.4925,0.3392,0.5088,B1,0.886552,26.2,2018-01-01,20180101-06-02,3,5.0
2,3-5-4,0,0.3692,0.5231,0.3135,0.4108,B2,2.836839,0,0.2714,...,0.4091,0.3472,0.4870,B1,0.580764,99.2,2018-01-01,20180101-06-03,3,5.0
3,1-5-2,0,0.4308,0.6308,0.3676,0.5514,A2,7.488113,0,0.3636,...,0.3939,0.3176,0.4706,B1,0.583266,16.8,2018-01-01,20180101-06-04,3,5.0
4,4-1-3,0,0.3636,0.5152,0.4469,0.5754,B1,6.454111,0,0.3231,...,0.4925,0.3916,0.5663,B2,0.470489,51.5,2018-01-01,20180101-06-05,3,5.0


In [10]:
testdf = pd.io.json.json_normalize(testLoadList)
testdf.head()

,funaken,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1rank,l1score,l2Fcnt,l2boat2r,...,l6boat3r,l6motor2r,l6motor3r,l6rank,l6score,odds,raceDate,raceId,raceWaveHeight,raceWindSpeed
0,1-5-2,0,0.3750,0.5714,0.3152,0.5163,B1,6.582412,0,0.3354,...,0.5723,0.3669,0.5089,B2,0.666744,40.6,2018-04-01,20180401-02-01,1,2.0
1,4-6-1,0,0.4034,0.5625,0.3297,0.4835,B1,7.479711,1,0.3247,...,0.4765,0.4934,0.5987,B1,1.950668,26.0,2018-04-01,20180401-02-02,1,2.0
2,3-2-5,0,0.4045,0.5955,0.3664,0.5954,B1,2.656783,0,0.3609,...,0.5465,0.3702,0.5240,A2,1.499352,9.3,2018-04-01,20180401-02-03,1,2.0
3,3-1-2,0,0.4121,0.5818,0.4138,0.5655,B1,8.429216,1,0.4653,...,0.5724,0.3627,0.5699,A2,2.021243,85.3,2018-04-01,20180401-02-04,1,2.0
4,3-4-2,0,0.3125,0.5125,0.3200,0.4400,B1,3.230705,1,0.3106,...,0.5284,0.4260,0.6036,B1,0.478606,15.9,2018-04-01,20180401-02-05,1,2.0


In [11]:
# 入力のデータ整形
xdf=df.drop(['funaken','odds','raceId','raceDate'],axis=1)
xdf=pd.get_dummies(xdf,columns=['l1rank','l2rank','l3rank','l4rank','l5rank','l6rank'])
#xdf.head()
xdf.describe()

# サンプル確認用
xdf_sample=xdf[0:5]

In [12]:
# 入力のデータ整形
testxdf=testdf.drop(['funaken','odds','raceId','raceDate'],axis=1)
testxdf=pd.get_dummies(testxdf,columns=['l1rank','l2rank','l3rank','l4rank','l5rank','l6rank'])
testxdf.head()

,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1score,l2Fcnt,l2boat2r,l2boat3r,l2motor2r,...,l4rank_B1,l4rank_B2,l5rank_A1,l5rank_A2,l5rank_B1,l5rank_B2,l6rank_A1,l6rank_A2,l6rank_B1,l6rank_B2
0,0,0.3750,0.5714,0.3152,0.5163,6.582412,0,0.3354,0.4939,0.3297,...,1,0,0,0,1,0,0,0,0,1
1,0,0.4034,0.5625,0.3297,0.4835,7.479711,1,0.3247,0.4675,0.3385,...,0,0,0,0,1,0,0,0,1,0
2,0,0.4045,0.5955,0.3664,0.5954,2.656783,0,0.3609,0.5030,0.3831,...,0,1,0,1,0,0,0,1,0,0
3,0,0.4121,0.5818,0.4138,0.5655,8.429216,1,0.4653,0.5972,0.3846,...,0,0,0,0,1,0,0,1,0,0
4,0,0.3125,0.5125,0.3200,0.4400,3.230705,1,0.3106,0.4783,0.3125,...,0,0,1,0,0,0,0,0,1,0


In [13]:
# 結果を整数化する
ydf=df['funaken']
ydf=pd.get_dummies(ydf,columns=['funaken'])
#ydf.head()
ydf.describe()

# サンプル確認用
ydf_sample=ydf[0:5]

In [14]:
# 重み付けのため、オッズのリストを作る
odf=pd.DataFrame(df['odds'])
odf.describe()

# サンプル確認用
odf_sample=odf[0:5]

In [15]:
print(type(df['odds']) )
print(type(odf))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [16]:
# 結果のOne-Hot表現を作る
testydf=testdf['funaken']
testydf=pd.get_dummies(testydf,columns=['funaken'])
testydf.head()

,1-2-3,1-2-4,1-2-5,1-2-6,1-3-2,1-3-4,1-3-5,1-3-6,1-4-2,1-4-3,...,6-3-4,6-3-5,6-4-1,6-4-2,6-4-3,6-4-5,6-5-1,6-5-2,6-5-3,6-5-4
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
#xAtrNum=62
#yclassNum=120

# create the model
## 保存しておく
#x = tf.placeholder(tf.float32,[None,xAtrNum])
#W = tf.Variable(tf.random_uniform([xAtrNum,yclassNum],-1.0,1.0))
#b = tf.Variable(tf.random_uniform([yclassNum],-1.0,1.0))
#y = tf.nn.softmax( (tf.matmul(x,W)+b) - tf.reduce_max(tf.matmul(x,W)+b)  )

In [18]:
xAtrNum=62
yclassNum=120

# create the model
## 保存しておく
#x = tf.placeholder(tf.float32,[None,xAtrNum])
#W = tf.Variable(tf.random_uniform([xAtrNum,yclassNum],-1.0,1.0))
#b = tf.Variable(tf.random_uniform([yclassNum],-1.0,1.0))
#y = tf.nn.softmax( (tf.matmul(x,W)+b) - tf.reduce_max(tf.matmul(x,W)+b)  )

# layerNumは2以上。
layerNum=2
layerClassNum=360*6
wInitMin= -0.01
wInitMax= 0.01
bInitMin= -0.0001
bInitMax= 0.0001

## 多層化
x = tf.placeholder(tf.float32,[None,xAtrNum])
x_hidden=[]
W_hidden=[]
b_hidden=[]

# 0 と最後については形が違うので個別に設定。
x_hidden.append(tf.Variable(tf.random_uniform([len(loadList),layerClassNum],-1.0,1.0)))
W_hidden.append(tf.Variable(tf.random_uniform([xAtrNum,layerClassNum],wInitMin,wInitMax)))
b_hidden.append(tf.Variable(tf.random_uniform([layerClassNum],bInitMin,bInitMax)))

for i in range(1,layerNum):
    x_hidden.append(tf.Variable(tf.random_uniform([len(loadList),layerClassNum],-1.0,1.0)))
    W_hidden.append(tf.Variable(tf.random_uniform([layerClassNum,layerClassNum],wInitMin,wInitMax)))
    b_hidden.append(tf.Variable(tf.random_uniform([layerClassNum],bInitMin,bInitMax)))

#x_hidden[0]=tf.sigmoid(tf.matmul(x,W_hidden[0])+b_hidden[0])
x_hidden[0]=tf.nn.elu(tf.matmul(x,W_hidden[0])+b_hidden[0])
for i in range(1,layerNum):
    #x_hidden[i]=tf.sigmoid(tf.matmul(x_hidden[i-1],W_hidden[i])+b_hidden[i])
    x_hidden[i]=tf.nn.elu(tf.matmul(x_hidden[i-1],W_hidden[i])+b_hidden[i])

# 0 と最後については形が違うので個別に設定。
W = tf.Variable(tf.random_uniform([layerClassNum,yclassNum],wInitMin,wInitMax))
b = tf.Variable(tf.random_uniform([yclassNum],bInitMin,bInitMax))
x_last=tf.nn.elu(tf.matmul(x_hidden[layerNum-1],W)+b)
y = tf.nn.softmax( x_last - tf.reduce_max(x_last) )
#y = tf.nn.softmax( tf.sigmoid(tf.matmul(x_hidden[layerNum-1],W)+b))


In [19]:
# define loss and optimizer
y_ = tf.placeholder(tf.float32,[None,yclassNum])
#cross_entropy=-tf.reduce_sum(y_*tf.log( tf.clip_by_value(y,1e-5,1e+5) ))
o_ = tf.placeholder(tf.float32,[None,1])
# 重み付き
cross_entropy=-tf.reduce_sum(y_*tf.log( tf.clip_by_value(y,1e-5,1e+5))*o_ )

# L2正則化
lambda_2=0.01
L2_sqr=tf.nn.l2_loss(W)
for i in range(layerNum):
    L2_sqr+=tf.nn.l2_loss(W_hidden[i])

loss=cross_entropy+lambda_2*L2_sqr

#cross_entropy=-tf.reduce_sum( (y_-y)**2 )
train_step=tf.train.GradientDescentOptimizer(0.0001).minimize(loss)

sess = tf.Session()


In [20]:
# tensorboard
with tf.name_scope('summary'):
    tf.summary.scalar('cross_entropy', cross_entropy)
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('/home/ec2-user/boatrace/log/tensorboard', sess.graph)

In [21]:
# train
maxEpochs=100

init = tf.global_variables_initializer()
sess.run(init)
for i in range(maxEpochs):
    sess.run(train_step,feed_dict={x:xdf.values,y_:ydf.values,o_:odf.values})
    if i % (maxEpochs/10) == 0:
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print("cross_entropy:"+str(sess.run(cross_entropy,feed_dict={x:xdf.values,y_:ydf.values,o_:odf.values}) ), \
              "loss:"+str(sess.run(loss,feed_dict={x:xdf.values,y_:ydf.values,o_:odf.values})),  \
              "accuracy:"+str(sess.run(accuracy, feed_dict={x: xdf.values, y_: ydf.values,o_:odf.values})) )
        #print("w:",sess.run(tf.reduce_sum(W),feed_dict={x: xdf.values, y_: ydf.values}))
        print("w:",sess.run(W,feed_dict={x: xdf.values, y_: ydf.values}))
        print("b:",sess.run(b,feed_dict={x:xdf.values,y_:ydf.values,o_:odf.values}))
        print("x_hidden:",sess.run(x_hidden[layerNum-1],feed_dict={x:xdf.values,y_:ydf.values,o_:odf.values}))
        print("x_last:",sess.run(x_last,feed_dict={x:xdf.values,y_:ydf.values,o_:odf.values}))
        print("Y:",sess.run(y ,feed_dict={x:xdf.values,y_:ydf.values,o_:odf.values}))
        print("W:",sess.run(tf.reduce_sum(W) ,feed_dict={x:xdf.values,y_:ydf.values,o_:odf.values}))
        for j in range(layerNum):
            print("X%d:" % (j),sess.run(tf.reduce_max(x_hidden[j]) ,feed_dict={x:xdf.values,y_:ydf.values,o_:odf.values}))
        #print("b:",sess.run(tf.reduce_sum(b) ,feed_dict={x:xdf.values,y_:ydf.values}))
print("sample:",sess.run(x_hidden[0],feed_dict={x:xdf_sample.values,y_:ydf_sample.values,o_:odf_sample.values}))
        

cross_entropy:8950172.0 loss:8950173.0 accuracy:0.0035587188
w: [[-0.00570614  0.00244553 -0.00193763 ...  0.00398657  0.00617642
   0.01294685]
 [-0.00679191  0.00598365  0.01301824 ... -0.02476747  0.00402582
   0.00498098]
 [ 0.0020573  -0.00685313 -0.00440134 ...  0.01318253  0.00426815
  -0.0019926 ]
 ...
 [-0.01058433  0.00313315  0.00395948 ... -0.00452402  0.0010217
   0.00902125]
 [ 0.00644954  0.00166077  0.0026663  ...  0.00434943 -0.00312494
  -0.01738071]
 [ 0.01602469  0.00381639  0.01026337 ...  0.02138362 -0.00924671
  -0.02472243]]
b: [ 0.5213556   0.1628332   0.26919922  0.34105986  0.46180636  0.3990372
  0.07096628  0.31872612  0.29973024  0.07157742  0.08878851  0.10736979
  0.36443013  0.06711273  0.20257488  0.16247049  0.6060077   0.14965367
  0.57107955 -0.09375276  0.39854875  0.40818456  0.12969519  0.28060246
  0.20820075  0.06988838  0.11548135 -0.06392184 -0.07982568  0.0804239
  0.07378203 -0.37715203 -0.11231761 -0.40483662 -0.36275202 -0.48217347
 -0.10

x_hidden: [[ 2.8143488e+04 -1.0000000e+00 -1.0000000e+00 ...  1.8500832e+04
  -1.0000000e+00 -1.0000000e+00]
 [ 1.2772119e+04 -1.0000000e+00 -1.0000000e+00 ...  8.3941074e+03
  -1.0000000e+00 -1.0000000e+00]
 [ 1.2951499e+04 -1.0000000e+00 -1.0000000e+00 ...  8.5114707e+03
  -1.0000000e+00 -1.0000000e+00]
 ...
 [ 1.9999559e+04 -1.0000000e+00 -1.0000000e+00 ...  1.3148097e+04
  -1.0000000e+00 -1.0000000e+00]
 [ 3.5890422e+04 -1.0000000e+00 -1.0000000e+00 ...  2.3595324e+04
  -1.0000000e+00 -1.0000000e+00]
 [ 5.4026426e+04 -1.0000000e+00 -1.0000000e+00 ...  3.5519043e+04
  -1.0000000e+00 -1.0000000e+00]]
x_last: [[-1.0000000e+00  7.5874010e+06  2.3328552e+06 ... -1.0000000e+00
   2.4475460e+06  9.7448162e+05]
 [-1.0000000e+00  3.4410945e+06  1.0579832e+06 ... -1.0000000e+00
   1.1100874e+06  4.4206234e+05]
 [-1.0000000e+00  3.4891380e+06  1.0727506e+06 ... -1.0000000e+00
   1.1255695e+06  4.4821947e+05]
 ...
 [-1.0000000e+00  5.3918170e+06  1.6577926e+06 ... -1.0000000e+00
   1.7393762e+

x_last: [[-1.0000000e+00  7.5869440e+06  2.3327148e+06 ... -1.0000000e+00
   2.4473988e+06  9.7442325e+05]
 [-1.0000000e+00  3.4408872e+06  1.0579194e+06 ... -1.0000000e+00
   1.1100204e+06  4.4203566e+05]
 [-1.0000000e+00  3.4889272e+06  1.0726860e+06 ... -1.0000000e+00
   1.1255016e+06  4.4819234e+05]
 ...
 [-1.0000000e+00  5.3914925e+06  1.6576931e+06 ... -1.0000000e+00
   1.7392712e+06  6.9254806e+05]
 [-1.0000000e+00  9.6770010e+06  2.9753538e+06 ... -1.0000000e+00
   3.1215925e+06  1.2428119e+06]
 [-1.0000000e+00  1.4568525e+07  4.4793535e+06 ... -1.0000000e+00
   4.6994090e+06  1.8709000e+06]]
Y: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
W: -6392.8154
X0: 1097.5062
X1: 524353.9
cross_entropy:19363478.0 loss:19398080.0 accuracy:0.005535785
w: [[-1.01978409e+00  3.60553414e-01  1.34611845e-01 ... -1.72833002e+00
   1.14845455e-01  2.86867563e-02]
 [ 3.69493276e-01 -1.1

Y: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
W: -6392.6875
X0: 1097.4835
X1: 524332.8
cross_entropy:19363478.0 loss:19398078.0 accuracy:0.005535785
w: [[-1.0197626e+00  3.6054626e-01  1.3460916e-01 ... -1.7282966e+00
   1.1484322e-01  2.8686197e-02]
 [ 3.6948612e-01 -1.1455895e-01 -2.2885857e-02 ...  6.3052332e-01
  -3.1936750e-02 -3.7981474e-04]
 [ 1.5293468e+00 -4.8126367e-01 -1.4934665e-01 ...  2.7031903e+00
  -1.1793640e-01 -2.5407985e-02]
 ...
 [-1.9442458e+00  6.6343546e-01  2.5768927e-01 ... -3.3055944e+00
   1.9001749e-01  3.9597861e-02]
 [-3.6384065e+00  1.1420805e+00  3.5908979e-01 ... -6.3946371e+00
   3.0449602e-01  3.9276410e-02]
 [ 1.6341560e+00 -4.9820489e-01 -1.4093085e-01 ...  2.8776593e+00
  -1.3846166e-01 -4.9467273e-02]]
b: [-2.11243439e+00  9.32671964e-01  4.61319745e-01 -8.28715742e-01
 -8.87870491e-01  4.60913479e-01  6.95727348e-01 -3.02009881e-01
  

X0: 1097.461
X1: 524311.8
cross_entropy:19363478.0 loss:19398078.0 accuracy:0.005535785
w: [[-1.01974118e+00  3.60539109e-01  1.34606481e-01 ... -1.72826326e+00
   1.14840984e-01  2.86856387e-02]
 [ 3.69478971e-01 -1.14556715e-01 -2.28854101e-02 ...  6.30510211e-01
  -3.19360793e-02 -3.79807170e-04]
 [ 1.52931583e+00 -4.81254131e-01 -1.49343669e-01 ...  2.70313787e+00
  -1.17934018e-01 -2.54074633e-02]
 ...
 [-1.94420767e+00  6.63422346e-01  2.57683903e-01 ... -3.30552769e+00
   1.90013617e-01  3.95970419e-02]
 [-3.63833499e+00  1.14205670e+00  3.59082639e-01 ... -6.39451313e+00
   3.04490060e-01  3.92755903e-02]
 [ 1.63412261e+00 -4.98194754e-01 -1.40928164e-01 ...  2.87760210e+00
  -1.38458982e-01 -4.94663045e-02]]
b: [-2.11243439e+00  9.32671964e-01  4.61319745e-01 -8.28715742e-01
 -8.87870491e-01  4.60913479e-01  6.95727348e-01 -3.02009881e-01
  5.12923598e-01  8.21127534e-01  6.16252899e-01  3.75957876e-01
  2.87176669e-01  9.34642434e-01  5.47710299e-01  5.30119479e-01
 -5.840074

In [22]:
print("sample:",sess.run(x_hidden[layerNum-1],feed_dict={x:xdf_sample.values,y_:ydf_sample.values,o_:odf_sample.values}))
#print("Y:",sess.run(y,feed_dict={x:xdf_sample.values,y_:ydf_sample.values,o_:odf_sample.values}))
print("Y:",sess.run(y,feed_dict={x:xdf.values,y_:ydf.values,o_:odf.values}))
print("x_last:",sess.run(y,feed_dict={x:xdf_sample.values,y_:ydf_sample.values,o_:odf_sample.values}))

sample: [[ 2.81390234e+04 -1.00000000e+00 -1.00000000e+00 ...  1.84978906e+04
  -1.00000000e+00 -1.00000000e+00]
 [ 1.27700859e+04 -1.00000000e+00 -1.00000000e+00 ...  8.39276758e+03
  -1.00000000e+00 -1.00000000e+00]
 [ 1.29494375e+04 -1.00000000e+00 -1.00000000e+00 ...  8.51011426e+03
  -1.00000000e+00 -1.00000000e+00]
 [ 1.62622832e+04 -1.00000000e+00 -1.00000000e+00 ...  1.06910039e+04
  -1.00000000e+00 -1.00000000e+00]
 [ 1.56655283e+04 -1.00000000e+00 -1.00000000e+00 ...  1.02979209e+04
  -1.00000000e+00 -1.00000000e+00]]
Y: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
x_last: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0

In [23]:
# evaluate trained_data
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: testxdf.values, y_: testydf.values}))


0.005169701


In [24]:
# メモリ使用チェック
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 1000: #ここだけアレンジ
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

|            Variable Name|    Memory|
 ------------------------------------ 
|                       df|  19096583|
|                 loadList|    200320|
|                      odf|    182192|
|             testLoadList|     38216|
|                   testdf|   3732815|
|                  testxdf|   1459376|
|                  testydf|    533984|
|                      xdf|   7465712|
|               xdf_sample|      1744|
|                      ydf|   2731424|
